In [1]:
import numpy as np
from qiskit.quantum_info import Statevector, partial_trace

# 상태 벡터 정의
state_vector = Statevector([0.3, 0.4, 0.5, 0.6])

# 첫 번째 큐빗에 대한 축소 밀도 행렬 계산
reduced_density_matrix_first = partial_trace(state_vector, [1])

# 두 번째 큐빗에 대한 축소 밀도 행렬 계산
reduced_density_matrix_second = partial_trace(state_vector, [0])

# 결과 출력
print("전체 밀도 행렬:")
print(state_vector)

print("\n첫 번째 큐빗의 축소 밀도 행렬:")
print(reduced_density_matrix_first)

print("\n두 번째 큐빗의 축소 밀도 행렬:")
print(reduced_density_matrix_second)


전체 밀도 행렬:
Statevector([0.3+0.j, 0.4+0.j, 0.5+0.j, 0.6+0.j],
            dims=(2, 2))

첫 번째 큐빗의 축소 밀도 행렬:
DensityMatrix([[0.34+0.j, 0.42+0.j],
               [0.42+0.j, 0.52+0.j]],
              dims=(2,))

두 번째 큐빗의 축소 밀도 행렬:
DensityMatrix([[0.25+0.j, 0.39+0.j],
               [0.39+0.j, 0.61+0.j]],
              dims=(2,))


In [2]:
import numpy as np
from qiskit.quantum_info import Statevector, partial_trace, Operator

# 상태 벡터 정의 및 정상화
state_vector = Statevector([0.3, 0.4, 0.5, 0.6, 0.2, 0.1, 0.3, 0.4])
print(state_vector)

# Pauli 행렬 정의
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
I = np.eye(2)

# 첫 번째 큐빗에 대한 축소 밀도 행렬 계산
reduced_density_matrix_first = partial_trace(state_vector, [0, 2]).data

# 세 번째 큐빗에 대한 축소 밀도 행렬 계산
reduced_density_matrix_third = partial_trace(state_vector, [0, 1]).data

# 첫 번째 큐빗에 X 연산 적용
first_qubit_result = np.trace(reduced_density_matrix_first @ X)

# 세 번째 큐빗에 Y 연산 적용
third_qubit_result = np.trace(reduced_density_matrix_third @ Y)

# 최종 결과 결합
final_result = first_qubit_result * third_qubit_result

# 결과 출력
print("첫 번째 큐빗에 대한 X 연산 결과:", first_qubit_result)
print("세 번째 큐빗에 대한 Y 연산 결과:", third_qubit_result)
print("최종 결과 (기대값):", final_result)


Statevector([0.3+0.j, 0.4+0.j, 0.5+0.j, 0.6+0.j, 0.2+0.j, 0.1+0.j, 0.3+0.j,
             0.4+0.j],
            dims=(2, 2, 2))
첫 번째 큐빗에 대한 X 연산 결과: (0.98+0j)
세 번째 큐빗에 대한 Y 연산 결과: 0j
최종 결과 (기대값): 0j


In [1]:
import numpy as np
from qiskit.quantum_info import Statevector, partial_trace

# 임의의 12 큐빗 상태 벡터 생성
np.random.seed(42)  # 랜덤 시드 설정
state_vector = np.random.rand(2**12) + 1j * np.random.rand(2**12)
state_vector = state_vector / np.linalg.norm(state_vector)  # 상태 벡터 정규화

# 상태 벡터를 Qiskit Statevector 객체로 변환
state_vector = Statevector(state_vector)

# Pauli 행렬 정의
Z = np.array([[1, 0], [0, -1]])
Y = np.array([[0, -1j], [1j, 0]])
I = np.eye(2)

# 관심 있는 큐빗을 제외하고 추적
rdm_indices = [i for i in range(12) if i not in [0, 3]]  # 추적할 큐빗 인덱스
reduced_density_matrix = partial_trace(state_vector, rdm_indices).data

# 축소 밀도 행렬과 Z 연산자의 기대값 계산
operator_reduced = np.kron(Y, Z)
expectation_value_reduced = np.trace(reduced_density_matrix @ operator_reduced)

# 결과 출력
print("첫 번째와 네 번째 큐빗의 Z 연산자에 대한 기대값 (축소 밀도 행렬):", expectation_value_reduced.real)

# 전체 행렬 계산
state_vector_ndarray = state_vector.data.reshape(-1, 1)

operator_full = Z
for i in range(1, 12):
    if i == 3:
        operator_full = np.kron(Y, operator_full)  # 네 번째 큐빗에 Z 연산자
    else:
        operator_full = np.kron(I, operator_full)  # 나머지 큐빗에 I 연산자

expectation_value_full = state_vector_ndarray.conj().T @ operator_full @ state_vector_ndarray

# 결과 출력
print(f"첫 번째와 네 번째 큐빗의 Z 연산자에 대한 기대값 (전체 행렬 계산): {expectation_value_full[0, 0].real:.019f}")

첫 번째와 네 번째 큐빗의 Z 연산자에 대한 기대값 (축소 밀도 행렬): -0.016615794593490153
첫 번째와 네 번째 큐빗의 Z 연산자에 대한 기대값 (전체 행렬 계산): -0.0166157945934902046


In [2]:
from functools import reduce
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector, partial_trace
from jqc.mapper.pauli import Pauli

def matrix(pauli):
    return reduce(np.kron, [p.matrix for p in pauli])

I = Pauli('I')
X = Pauli('X')
Y = Pauli('Y')
Z = Pauli('Z')

p_string = (Y, Z, Z, Z, Z, Z, Z, Z, Y, I, I, X, Z, Z, Z, Z, X, I, I, I, I, I)
qc = QuantumCircuit(len(p_string))
qc.x(0)
qc.x(1)
qc.x(len(p_string)//2)
qc.x(len(p_string)//2 + 1)
state_vector = Statevector.from_instruction(qc)
reduce_idx = [idx for idx, p in enumerate(p_string) if p.symbol == 'I']
left_pauli = [p for p in p_string if p.symbol != 'I']
rdm = partial_trace(state_vector, reduce_idx).data

op = matrix(left_pauli)

# trace = np.trace(rdm @ op)
# print(trace)

: 